In [1]:
%%capture
# special kaggle workspace command

!pip install deep-translator
!pip install spacy

In [2]:
import numpy as np
import pandas as pd

import spacy
import en_core_web_sm, en_core_web_lg

from deep_translator import GoogleTranslator

from transformers import pipeline

2024-05-14 12:27:16.063808: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-14 12:27:16.064022: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-14 12:27:16.234432: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Задача 3. Аналіз семантики товарів

У нашій базі є інформація про числові оцінки товару з текстовим коментарем. Чув, що сучасні алгоритми вміють читати та розуміти ці відгуки. Давайте реалізуємо функціонал для класифікації позитивних/негативних коментарів та зведемо аналітику. Мене цікавить кореляція текстових коментарів від числової оцінки (1-5), товари з найкращими/найгіршими відгуками, продавці — які збирають лише негатив.

Також хотілося б виділити у тексті коментаря таку сутність, як ціна — щоб аналітичний відділ переглядав цю інформацію на предмет наявності цін конкурентів.

Як результат: Візуальна репрезентація результатів (.ipynb файл з візауалізаціями або BI-звіт) аналітики та код, який її сформував. А також, код, який може приймати текстовий коментар на вхід і оцінювати семантику як позитивну, негативну або нейтральну.

In [3]:
FOLDER = '/kaggle/input/test-assigment-smart-it/' # change to your folder with data

order_reviews = pd.read_csv(FOLDER + 'order_reviews.csv')
order_items = pd.read_csv(FOLDER + 'order_items.csv')

In [4]:
order_reviews.isna().sum()

review_id                      0
order_id                       0
review_score                   0
review_comment_title       88285
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64

## Concatenate comment title and message

In [5]:
order_reviews[['review_comment_title', 'review_comment_message']] = order_reviews[['review_comment_title', 'review_comment_message']].fillna(value='')

order_reviews['full_review'] = order_reviews['review_comment_title'] + ' ' + order_reviews['review_comment_message']

order_reviews.head(5)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,full_review
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,,,2018-01-18 00:00:00,2018-01-18 21:46:59,
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,,,2018-03-10 00:00:00,2018-03-11 03:05:13,
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,,,2018-02-17 00:00:00,2018-02-18 14:36:24,
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,Recebi bem antes do prazo estipulado.
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53,Parabéns lojas lannister adorei comprar pela ...


## Add seller_id to order reviews

In [6]:
unique_order_seller = order_items[['order_id', 'seller_id']].drop_duplicates(subset='order_id').set_index('order_id')

for i, row in order_reviews.iterrows():
    try:
        # in case order_items doesnt have order_id from order_reviews.csv
        seller = unique_order_seller.loc[row['order_id'], 'seller_id']
        order_reviews.at[i, 'seller_id'] = seller
    except KeyError as e:
        order_reviews.at[i, 'seller_id'] = None

In [7]:
print(order_reviews.shape)

order_reviews.isna().sum()

(100000, 9)


review_id                    0
order_id                     0
review_score                 0
review_comment_title         0
review_comment_message       0
review_creation_date         0
review_answer_timestamp      0
full_review                  0
seller_id                  778
dtype: int64

## Drop empty comments and titles

In [8]:
dropped_order_reviews = order_reviews[order_reviews['full_review'] != ' ']
dropped_order_reviews = dropped_order_reviews.drop(dropped_order_reviews[dropped_order_reviews['seller_id'].isna()].index, axis=0)

dropped_order_reviews.shape

(42933, 9)

In [9]:
dropped_order_reviews.sample(5, random_state=1)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,full_review,seller_id
65895,e0b3aaaeaa777cdb311085a744ee64d4,a835505a16cefe4fe794476d79374f81,1,,RECEBI O PRODUTO ERRADO. CADE O QUE COMPREI?????,2018-08-21 00:00:00,2018-08-24 21:27:43,RECEBI O PRODUTO ERRADO. CADE O QUE COMPREI?????,f8db351d8c4c4c22c6835c19a46f01b0
52656,4af38a85bffd33de3992d1bbb3a48f4e,2462994fc67121ce4376d1c6baf63388,5,Ótimo,"Muito bom o produto é chegou rápido, recomendo!",2018-06-24 00:00:00,2018-06-24 21:17:21,"Ótimo Muito bom o produto é chegou rápido, rec...",4869f7a5dfa277a7dca6462dcf3b52b2
97557,c1b984637182727734bb8919301856e4,fbd05af8c1751b972fc6e19b1190990e,5,,Comprei pq confio no site do stark,2017-02-22 00:00:00,2017-02-28 03:33:37,Comprei pq confio no site do stark,0c8380b62e38e8a1e6adbeba7eb9688c
50177,0ad32ff8572b160bd0e4dd671d905be8,c6a9e39db5f61068d017ee903b277478,5,,Entregou antes do esperado 🤗😍,2017-08-17 00:00:00,2017-08-18 15:34:28,Entregou antes do esperado 🤗😍,cc419e0650a3c5ba77189a1882b7556a
40986,784f60a86ba6acd694a679ee35e5ce10,676e40056867189a1b3599f3e646f421,1,,Deveriam ter feito contato.,2017-05-05 00:00:00,2017-05-11 19:57:56,Deveriam ter feito contato.,38e679b9e0064cd94c6f035707344dae


# Translate from portugal to english for classification model

For test or learning purpose I find ready-to-use english classification review model trained 400,000 samples of Amazon reviews. Becauese it was trained on Amazon reviews (its basically english platform) this model scores bad portugal and good translated comments. (It is demonstrated some cells below.)

While It is possible to classify portugal or other languages, but in this case we had to find, or create own dataset with portugal reviews and fit it to language model.

In [10]:
translator = GoogleTranslator(source='pt', target='en')

dropped_order_reviews.insert(dropped_order_reviews.shape[1], 'translated_full_review', '')

### !!! Translation takes time. For test purposes, I speed up run time by reducing rows to translate.

In [11]:
%%time 
# special kaggle workspace command


N_ROWS_TO_PROCESS = 2000


for n_iterrations, o in enumerate(dropped_order_reviews.iterrows()):
    if n_iterrations == N_ROWS_TO_PROCESS:
        break
    
    i, row = o
    
    translated_full_review = translator.translate(row['full_review'])
    dropped_order_reviews.at[i, 'translated_full_review'] = translated_full_review
    

# save
dropped_order_reviews.to_csv('translated_order_reviews.csv')

CPU times: user 41.1 s, sys: 1.87 s, total: 43 s
Wall time: 30min 3s


In [12]:
dropped_order_reviews.head(3)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,full_review,seller_id,translated_full_review
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,Recebi bem antes do prazo estipulado.,a1043bafd471dff536d0c462352beb48,I received it well before the stipulated deadl...
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53,Parabéns lojas lannister adorei comprar pela ...,989becdce12ebc39863c2bceab6f3ca1,Congratulations lannister stores I loved shopp...
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelh...,2018-05-22 00:00:00,2018-05-23 16:45:47,recomendo aparelho eficiente. no site a marca ...,05ff92fedb5be47920fea08e501238b9,I recommend an efficient device. On the websit...


## Leave only reviews that we translated

In [13]:
only_with_translated_reviews = dropped_order_reviews[dropped_order_reviews['translated_full_review'] != '']
only_with_translated_reviews.insert(only_with_translated_reviews.shape[1], 'classification_rate', 0)
only_with_translated_reviews.shape

(2000, 11)

# Classify comments

In [14]:
classification_pipeline = pipeline('text-classification', model='AdamCodd/distilbert-base-uncased-finetuned-sentiment-amazon')

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

## See how classifiaction model perfomrs on portugal and translated to eng comments

In [15]:
negative_review = order_reviews.loc[54584, 'review_comment_message']   # negative
negative_review_2 = order_reviews.loc[24307, 'review_comment_message'] # negative, kind of neutral
positive_review = order_reviews.loc[30206, 'review_comment_message']   # positive
neutral_review = order_reviews.loc[32267, 'review_comment_message']    # mostly negative
neutral_review_2 = order_reviews.loc[84119, 'review_comment_message']  # more positive

In [16]:
test_reviews = [negative_review,
                negative_review_2,
                positive_review,
                neutral_review,
                neutral_review_2]

print('This is scores for portugal comments')
classification_pipeline(test_reviews)

This is scores for portugal comments


[{'label': 'negative', 'score': 0.5691413283348083},
 {'label': 'positive', 'score': 0.5894104242324829},
 {'label': 'negative', 'score': 0.9652561545372009},
 {'label': 'negative', 'score': 0.7331950068473816},
 {'label': 'positive', 'score': 0.5284745097160339}]

In [17]:
translated_test_reviews = [translator.translate(r) for r in test_reviews]

print('This is scores for translated to english comments')
classification_pipeline(translated_test_reviews)

This is scores for translated to english comments


[{'label': 'negative', 'score': 0.9244617223739624},
 {'label': 'negative', 'score': 0.6676163673400879},
 {'label': 'positive', 'score': 0.99666827917099},
 {'label': 'negative', 'score': 0.7776234149932861},
 {'label': 'positive', 'score': 0.8268204927444458}]

## Classifing only translated comments. Positve is 5, negative is 1 and neutral is 3

In [18]:
def classify_review(review):
    if type(review) is not str:
        return 3
    
    classification_dict = classification_pipeline(review)[0]
    score = classification_dict['score']
    label = classification_dict['label']
    
    classification_rate = 3
    if score < 0.6: 
        classification_rate = 3
    elif label == 'positive':
        classification_rate = 5
    elif label == 'negative':
        classification_rate = 1
    
    return classification_rate

In [19]:
for i, row in only_with_translated_reviews.iterrows():
    translated_full_review = row['translated_full_review']
    
    rate = classify_review(translated_full_review)
    
    only_with_translated_reviews.at[i, 'classification_rate'] = rate

In [20]:
only_with_translated_reviews.tail()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,full_review,seller_id,translated_full_review,classification_rate
4618,fdd899c25072756eb4fbbcb21ad974a4,6b21d4143acd1caff8e127e69aba3881,5,,N/c,2017-03-19 00:00:00,2017-03-19 07:34:44,N/c,6560211a19b47992c3666cc44a7e94c0,N/c,1
4619,f3c8b0799585edd96736e485ef42f264,32e839df202e265a92fc882a73aca02f,5,,Parabéns,2017-12-08 00:00:00,2017-12-09 02:26:55,Parabéns,1f50f920176fa81dab994f9023523100,Congratulations,5
4620,fcaccf956b3b8b0b7f30b13712ed6037,9c2cd65f37fcbe73d562581e5d53577b,4,,"Mochila ""simples"" (com os dois compartimentos ...",2017-07-11 00:00:00,2017-07-12 15:10:59,"Mochila ""simples"" (com os dois compartimentos...",6cd68b3ed6d59aaa9fece558ad360c0a,"""Simple"" backpack (with the two standard compa...",5
4621,ca00693cc34e3e80ca90494237cc29a6,452b706af12e25e6c8c740b4dc493ab0,3,,produto chegou em perfeito estado e com antece...,2017-01-18 00:00:00,2017-01-20 23:55:07,produto chegou em perfeito estado e com antec...,4a3ca9315b744ce9f8e9374361493884,Product arrived in perfect condition and in ad...,5
4623,9b9d1634c1dd596afeb30fc03f53253f,52b72b336dbd155b3d75bb3fcc799ac3,5,,Gostei muito é recomendo para todos essa não é...,2017-07-27 00:00:00,2017-07-27 23:31:15,Gostei muito é recomendo para todos essa não ...,6560211a19b47992c3666cc44a7e94c0,I really liked it and I recommend it to everyo...,5


## Compare average user rates with classified comments rates

In [21]:
avg_review_score = sum(only_with_translated_reviews['review_score']) / only_with_translated_reviews.shape[0]
print('Average review score by user is', avg_review_score)

avg_classified_comment_score = sum(only_with_translated_reviews['classification_rate']) / only_with_translated_reviews.shape[0]
print('Average review score by classified comments rate is', avg_classified_comment_score)

print(f'Ratio classified comments rate to user rate(review_score column) is {(avg_classified_comment_score/avg_review_score):.3f}')

Average review score by user is 3.728
Average review score by classified comments rate is 3.663
Ratio classified comments rate to user rate(review_score column) is 0.983


## Worst sellers by classified comments rates

In [22]:
avg_classified_rate_seller = only_with_translated_reviews.groupby('seller_id')['classification_rate'].mean().sort_values()
avg_classified_rate_seller[:25]

seller_id
adbc26658d6c7b4b6219f9d934598091    1.0
5c6d4016c2a288f074fa0848a2b653d2    1.0
5bc55dbe2f12b6af6d83ed46023e0dc8    1.0
eddd1991bd6d9309995a480c39b95137    1.0
2089a6d640999f9b9141ac719b2af596    1.0
5bc24d989e71e93c33e50a7782431b0e    1.0
5b67882648febfb6e1225e6ad7164f6b    1.0
5b179e9e8cc7ab6fd113a46ca584da81    1.0
5a93f3ab0ef4c84ed5e1b5dbf23978bc    1.0
cfb1a033743668a192316f3c6d1d2671    1.0
d05ae8f7a5bd1d2a690a44cd079e4e27    1.0
ed8cb7b190ceb6067227478e48cf8dde    1.0
ed4acab38528488b65a9a9c603ff024a    1.0
575df70bde3f9f2b30bf8d2e9910d725    1.0
23a0ffca143a15b9b5bbdb7d552f626a    1.0
ecccfa2bb93b34a3bf033cc5d1dcdc69    1.0
56642bcb79900e777d68e91915cb4267    1.0
557f22c76691849db52d2abccf0015d0    1.0
5d0363b33554b373851fc1622e4d5f3c    1.0
ce27a3cc3c8cc1ea79d11e561e9bebb6    1.0
6039e27294dc75811c0d8a39069f52c0    1.0
eebb3372362aa9a46975164bed19a7e7    1.0
c990d6cf976a5718aaedc539f383ab88    1.0
6e386f64c84e482f0b9abf960797ee1b    1.0
ca3bd7cd9f149df75950150d010fe4

# Find entity price in comments

In [23]:
nlp = en_core_web_lg.load()

Considering tune model for better entity recogntion. See below two examples

In [24]:
test_entity = nlp('It costs a lot, 5 dollars')
test_entity_2 = nlp('another shop "Name" has better price')

print([(e.text, e.label_) for e in test_entity.ents])
print([(e.text, e.label_) for e in test_entity_2.ents])

[('5 dollars', 'MONEY')]
[]


## Make column 'is_price_in_comment'

In [25]:
only_with_translated_reviews.insert(only_with_translated_reviews.shape[1], 'is_price_in_comment', False)


for i, row in only_with_translated_reviews.iterrows():
    translated_full_review = row['translated_full_review']
    
    if type(translated_full_review) is not str:
        continue
    
    processed_text = nlp(translated_full_review)
    labels_in_comment = [e.label_ for e in processed_text.ents]
    
    if 'MONEY' in labels_in_comment:
        only_with_translated_reviews.at[i, 'is_price_in_comment'] = True

## See results

In [26]:
print(f"Number of comment with price entity is { (only_with_translated_reviews['is_price_in_comment']==True).sum() }\nTotal length of translated and classified rows is {only_with_translated_reviews.shape[0]}")

Number of comment with price entity is 6
Total length of translated and classified rows is 2000


A few examples below

In [27]:
only_with_translated_reviews[only_with_translated_reviews['is_price_in_comment']==True][:2]

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,full_review,seller_id,translated_full_review,classification_rate,is_price_in_comment
115,ae728c1061bf163b4bd256ad9ee0bb83,45c780334bc32cb77559a65c5f171160,1,,Este foi o pedido\r\nBalde Com 128 Peças - Blo...,2018-01-04 00:00:00,2018-01-10 09:52:57,Este foi o pedido\r\nBalde Com 128 Peças - Bl...,8cbac7e12637ed9cffa18c7875207478,This was the request\r\nBucket with 128 Pieces...,1,True
334,b396ba75350276a6cc1993b6627dceea,43050a51a257b17d51295e07650cb654,1,Lamenetável...,"Boa Noite\r\n\r\né lamentável, esta loja que t...",2018-08-23 00:00:00,2018-08-30 03:02:39,"Lamenetável... Boa Noite\r\n\r\né lamentável, ...",6560211a19b47992c3666cc44a7e94c0,"Pitiful... Good Night\r\n\r\nIt's regrettable,...",1,True


In [28]:
price_entity_com_1 = only_with_translated_reviews.loc[115, 'translated_full_review']
price_entity_com_2 = only_with_translated_reviews.loc[334, 'translated_full_review']

print(f"A few examples of comments with recognited price:\n\n'{price_entity_com_1}'\n\n'{price_entity_com_2}'")

A few examples of comments with recognited price:

'This was the request
Bucket with 128 Pieces - Building Blocks 2 units - R$ 25.00 each (NOT DELIVERED)
Sold and delivered targaryen
Eva Rug No. Letters 36 Pieces Children 1 unit - R$ 35.90 (THIS WAS DELIVERED'

'Pitiful... Good Night

It's regrettable, this store I've made so many purchases and today doesn't even give me the satisfaction of a watch worth 50.00 reais... a defective product and until now it hasn't given me a response'
